In [ ]:
import numpy as np
import requests
from abc import ABC, abstractmethod
from dataclasses import dataclass
import datetime
from typing import List, Tuple

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
@dataclass
class EtfPoint:

    price: float
    date: datetime.date = datetime.date(1900, 1, 1)

@dataclass
class EtfPlotInfo:
    name: str
    prices: List[float]

In [ ]:
class EtfSource(ABC):

    def __init__(self, urlMappings: dict[str, str]):
        self.urlMapping = urlMappings

    @abstractmethod
    def createPoint(self, item: dict[str, str]):
        pass

    def get(self, etfName: str) -> List[EtfPoint]:
        url: str = self.urlMapping[etfName]
        
        items: List[dict[str, str]] = requests.get(url).json()

        return list(map(self.createPoint, items))

In [ ]:
class BrdSource(EtfSource):

    def createPoint(self, item: dict[str, str]):
        price = float(item['vuan'])
        date = item['data'].split('-')
        year = int(date[0])
        month = int(date[1])
        day = int(date[2])

        return EtfPoint(price, datetime.date(year, month, day))

    def __init__(self):
        idMappings = {
            'GLOBAL_E': '10',
            'GLOBAL_A': '6'
        }

        urlMappings = {
            name : 'https://www.brdam.ro/fdata/graph/{0}/all'.format(idMappings[name]) for name in idMappings
        }

        super().__init__(urlMappings)

class AllianzSource(EtfSource):

    def createPoint(self, item: dict[str, str]):
        price = float(item['FundPrice'])
        date = (item['Date'].split('T')[0]).split('-')
        year = int(date[0])
        month = int(date[1])
        day = int(date[2])
        
        return EtfPoint(price, datetime.date(year, month, day))

    def __init__(self):
        idMappings = {
            'EUROPE_EQUITY': '27',
            'WORLD_EQUITY': '26'
        }

        urlMappings = {
            name: 'https://mobil.allianztiriac.ro/api/financial/investmentfund/{0}?since=1900-01-01&until=2100-01-01'.format(idMappings[name]) for name in idMappings
        }

        super().__init__(urlMappings)

In [ ]:
class EtfInterpolator:

    def _interpolatePrices(self):
        self._prices: dict[datetime.date, float] = {}

        if len(self._points) == 0:
            return
        elif len(self._points) == 1:
            self._prices[self._points[0].date] = self._points[0].price
            return

        last_point = self._points[0]
        self._prices[last_point.date] = last_point.price

        for i in range(1, len(self._points)):
            next_point = self._points[i]

            ndays = (next_point.date - last_point.date).days
            for delta in range(1, ndays + 1):
                interpDate = last_point.date + datetime.timedelta(days=delta)

                interpFactor = delta / ndays
                interpPrice = (1 - interpFactor) * last_point.price + interpFactor * next_point.price

                self._prices[interpDate] = interpPrice

            last_point = next_point

    def __init__(self, points: List[EtfPoint], interpType: str = 'linear'):
        points.sort(key=lambda x: x.date)
        
        self._points = points
        self._minDate = points[0].date
        self._maxDate = points[-1].date
        self._interpType = interpType

        self._interpolatePrices()

    def getMin(self):
        return self._minDate

    def getMax(self):
        return self._maxDate

    def getPrice(self, date: datetime.date):
        if not date in self._prices:
            raise ValueError('Date out of range')

        return self._prices[date]

    def getRange(self, startDate: datetime.date = None, endDate: datetime.date = None, 
            step: int = 1, normalizedStart = False, emaWeight: float = 0.0):
        if startDate is None:
            startDate = self._minDate

        if endDate is None:
            endDate = self._maxDate

        if not (startDate in self._prices and endDate in self._prices):
            raise ValueError('Dates out of range')
            
        pointRange: List[EtfPoint] = []

        ndays = (endDate - startDate).days

        if normalizedStart:
            weightedPrice = 1.0
        else:
            weightedPrice = self._prices[startDate]

        for delta in range(0, ndays + 1, step):
            date  = startDate + datetime.timedelta(days=delta)
            price = self._prices[date]

            if normalizedStart:
                price = price / self._prices[startDate]

            weightedPrice = weightedPrice * emaWeight + price * (1 - emaWeight)

            pointRange.append(EtfPoint(weightedPrice, date))
        
        return pointRange

def largestCommonInterval(*interps: List[EtfInterpolator]) -> Tuple[datetime.date, datetime.date]:
    minDate = interps[0].getMin()
    maxDate = interps[0].getMax()

    for i in range(1, len(interps)):
        interp = interps[i]

        if minDate < interp.getMin():
            minDate = interp.getMin()
        
        if maxDate > interp.getMax():
            maxDate = interp.getMax()

    return (minDate, maxDate)

In [ ]:
def plotPrices(dates: List[datetime.date], etfs: List[EtfPlotInfo]):
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d/%m/%Y'))
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=365))

    for etf in etfs:
        assert len(dates) == len(etf.prices)
        plt.plot(dates, etf.prices, label=etf.name)

    plt.legend()

    plt.gcf().autofmt_xdate()

def plotGains(gains: List[Tuple[datetime.date, float]]):
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%m/%Y'))
    plt.gca().xaxis.set_major_locator(mdates.YearLocator())

    dates = list(map(lambda point: point[0], gains))
    prices = list(map(lambda point: point[1] * 100, gains))

    plt.bar(dates, prices, width=20)
    plt.gcf().autofmt_xdate()
    

def listToDict(points: List[EtfPoint]):
    return {
        point.date : point.price for point in points
    }

def periodicGains(points: List[EtfPoint], period: datetime.timedelta, startDate = None) -> List[Tuple[datetime.date, float]]:
    pricesDict = listToDict(points)
    maxDate = points[-1].date

    if startDate is None:
        startDate = points[0].date

    if not startDate in pricesDict:
        raise ValueError('Start date not in range')

    periodicGains: List[Tuple[datetime.date, float]] = []

    lastDate = startDate
    while lastDate in pricesDict:
        nextDate = lastDate + period

        if not nextDate in pricesDict:
            nextDate = maxDate

            if nextDate == lastDate:
                break

        priceRatio = pricesDict[nextDate] / pricesDict[lastDate]
        periodicGains.append((lastDate, priceRatio - 1.0))

        lastDate = nextDate

    return periodicGains

In [ ]:
brds = BrdSource()
alls = AllianzSource()

global_e = EtfInterpolator(brds.get('GLOBAL_E'))
global_a = EtfInterpolator(brds.get('GLOBAL_A'))
world_equity = EtfInterpolator(alls.get('WORLD_EQUITY'))
europe_equity = EtfInterpolator(alls.get('EUROPE_EQUITY'))

In [ ]:
etfs = []

minDate, maxDate = largestCommonInterval(global_a, world_equity, europe_equity)

print(minDate, maxDate)
emaWeight = 0.95

#pointsList = global_e.getRange(minDate, maxDate, normalizedStart=True, emaWeight=emaWeight)
#dates = list(map(lambda point: point.date, pointsList))
#prices = list(map(lambda point: point.price, pointsList))
#etfs.append(EtfPlotInfo('BRD - Global E', prices))

pointsList = global_a.getRange(minDate, maxDate, normalizedStart=True, emaWeight=emaWeight)
dates = list(map(lambda point: point.date, pointsList))
prices = list(map(lambda point: point.price, pointsList))
etfs.append(EtfPlotInfo('BRD - Global A', prices))

pointsList = europe_equity.getRange(minDate, maxDate, normalizedStart=True, emaWeight=emaWeight)
dates = list(map(lambda point: point.date, pointsList))
prices = list(map(lambda point: point.price, pointsList))
etfs.append(EtfPlotInfo('Allianz - Europe Equity', prices))

pointsList = world_equity.getRange(minDate, maxDate, normalizedStart=True, emaWeight=emaWeight)
dates = list(map(lambda point: point.date, pointsList))
prices = list(map(lambda point: point.price, pointsList))
etfs.append(EtfPlotInfo('Allianz - World Equity', prices))

plotPrices(dates, etfs)

In [ ]:
emaWeight = 0.98

#pointsList = global_e.getRange(normalizedStart=True, emaWeight=emaWeight)
#global_e_gains = periodicGains(pointsList, datetime.timedelta(days=365))
#print(global_e_gains)

startDate = datetime.date(2013, 1, 1)
delta = datetime.timedelta(days=30)

pointsList = global_a.getRange(normalizedStart=True, emaWeight=emaWeight, startDate=startDate)
global_a_gains = periodicGains(pointsList, delta)

pointsList = world_equity.getRange(normalizedStart=True, emaWeight=emaWeight, startDate=startDate)
world_equity_gains = periodicGains(pointsList, delta)

pointsList = europe_equity.getRange(normalizedStart=True, emaWeight=emaWeight, startDate=startDate)
europe_equity_gains = periodicGains(pointsList, delta)

In [ ]:
plotGains(world_equity_gains)

In [ ]:
plotGains(global_a_gains)

In [ ]:
plotGains(europe_equity_gains)